In [47]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [23]:
X_balance_scale = pd.read_csv('data/X_balance_scale', delimiter=',')
y_balance_scale = pd.read_csv('data/y_balance_Scale', delimiter=',')

X_car_eval = pd.read_csv('data/X_car_eval', delimiter=',')
y_car_eval = pd.read_csv('data/y_car_eval', delimiter=',')

X_contraceptive = pd.read_csv('data/X_contraceptive', delimiter=',')
y_contraceptive = pd.read_csv('data/y_contraceptive', delimiter=',')

X_dermatology = pd.read_csv('data/X_dermatology', delimiter=',')
y_dermatology = pd.read_csv('data/y_dermatology', delimiter=',')

X_glass = pd.read_csv('data/X_glass', delimiter=',')
y_glass = pd.read_csv('data/y_glass', delimiter=',')

X_hayes = pd.read_csv('data/X_hayes', delimiter=',')
y_hayes = pd.read_csv('data/y_hayes', delimiter=',')

X_heart = pd.read_csv('data/X_heart', delimiter=',')
y_heart = pd.read_csv('data/y_heart', delimiter=',')

X_new_thyroid = pd.read_csv('data/X_new_thyroid', delimiter=',')
y_new_thyroid = pd.read_csv('data/y_new_thyroid', delimiter=',')

X_page = pd.read_csv('data/X_page', delimiter=',')
y_page = pd.read_csv('data/y_page', delimiter=',')

X_pen_based = pd.read_csv('data/X_pen_based', delimiter=',')
y_pen_based = pd.read_csv('data/y_pen_based', delimiter=',')

X_shuttle = pd.read_csv('data/X_shuttle', delimiter=',')
y_shuttle = pd.read_csv('data/y_shuttle', delimiter=',')

X_vertebra = pd.read_csv('data/X_vertebra', delimiter=',')
y_vertebra = pd.read_csv('data/y_vertebra', delimiter=',')

X_wine = pd.read_csv('data/X_wine', delimiter=',')
y_wine = pd.read_csv('data/y_wine', delimiter=',')

X_yeast = pd.read_csv('data/X_yeast', delimiter=',')
y_yeast = pd.read_csv('data/y_yeast', delimiter=',')

X_fars = pd.read_csv('data/X_fars', delimiter=',')
y_fars = pd.read_csv('data/y_fars', delimiter=',')

In [49]:
X_list = [
    X_balance_scale, X_car_eval, X_contraceptive,
    X_dermatology, X_glass, X_hayes, X_heart,
    X_new_thyroid, X_page, X_pen_based, X_shuttle,
    X_vertebra, X_wine, X_yeast, X_fars
]

y_list = [
    y_balance_scale, y_car_eval, y_contraceptive,
    y_dermatology, y_glass, y_hayes, y_heart,
    y_new_thyroid, y_page, y_pen_based, y_shuttle,
    y_vertebra, y_wine, y_yeast, y_fars
]

name_list = [
    'Balance Scale', 'Car Evaluation', 'Contraceptive',
    'Dermatology', 'Glass', 'Hayes', 'Heart',
    'New Thyroid', 'Page', 'Pen Based', 'Shuttle',
    'Vertebra', 'Wine', 'Yeast', 'FARS'
]

# Prediction

In [45]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from utils import *
from tqdm import tqdm

In [50]:
clf_dict = {
    'AdaBoost' : AdaBoostClassifier(n_estimators=100, random_state=42),
    'GradientBoosting' : GradientBoostingClassifier(n_estimators=100, learning_rate=0.3, max_features=2, max_depth=5, random_state=42)
}

clf_score_dict = {}

In [51]:
for clf_name, clf in tqdm(clf_dict.items()):
    score = {}
    
    for i, (X, y) in enumerate(zip(X_list, y_list)):

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        clf.fit(X_train, y_train) 
        
        y_preds = clf.predict(X_test) 
        y_scores = clf.predict_proba(X_test)

        score_p = metric_list(y_test,y_preds,y_scores, len(y.iloc[:,0].unique()))
        score[name_list[i]]= score_p
    
    df = pd.DataFrame.from_dict(score, orient='index', columns=['accuracy', 'precision', 'recall', 'f1','Macro-Averaged AUPRC',
                                                            'F_measure Beta=10','Modified mcc','MCC metirc','Gmean'])
    clf_score_dict[clf_name] = df

100%|██████████| 2/2 [01:36<00:00, 48.15s/it]


In [53]:
clf_score_dict['AdaBoost']

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Balance Scale,0.944000,0.965778,0.944000,0.949436,0.697356,0.945939,0.882886,0.910854,0.881972
Car Evaluation,0.800578,0.801116,0.800578,0.778251,0.650552,0.800627,0.603533,0.573091,0.691205
Contraceptive,0.589831,0.596148,0.589831,0.585984,0.502358,0.590399,0.358602,0.365253,0.233743
Dermatology,0.851351,0.784459,0.851351,0.805219,0.803922,0.844802,0.778439,0.819744,0.000000
Glass,0.488372,0.481190,0.488372,0.425292,0.293444,0.487710,0.175065,0.326144,0.000000
Hayes,0.500000,0.307692,0.500000,0.361111,0.833333,0.473118,0.441292,0.486427,1.000000
Heart,0.508197,0.436690,0.508197,0.464272,0.318955,0.500743,0.245199,0.264758,0.000000
New Thyroid,0.720930,0.618605,0.720930,0.633926,0.653704,0.710250,0.331742,0.401677,0.000000
Page,0.738813,0.915992,0.738813,0.794022,0.676391,0.752037,0.517763,0.421373,0.000000
Pen Based,0.283765,0.243578,0.283765,0.170141,0.199340,0.279572,0.212055,0.256988,0.000000


In [54]:
clf_score_dict['GradientBoosting']

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Balance Scale,0.816000,0.787066,0.816000,0.798012,0.697514,0.813282,0.492666,0.679347,0.800308
Car Evaluation,0.965318,0.973694,0.965318,0.967669,0.973750,0.966073,0.916324,0.928579,0.748663
Contraceptive,0.552542,0.554383,0.552542,0.549183,0.561179,0.552709,0.300376,0.300456,0.198232
Dermatology,0.986486,0.987838,0.986486,0.986392,1.000000,0.986609,0.980653,0.982152,0.875000
Glass,0.883721,0.893113,0.883721,0.877415,0.287789,0.884567,0.749597,0.852748,0.000000
Hayes,0.843750,0.865385,0.843750,0.845858,0.956776,0.845672,0.792569,0.764506,0.900000
Heart,0.491803,0.379372,0.491803,0.425061,0.376057,0.478901,0.191558,0.204533,0.000000
New Thyroid,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Page,0.715068,0.893834,0.715068,0.793868,0.370869,0.728310,0.264130,0.253892,0.000000
Pen Based,0.994088,0.994117,0.994088,0.994086,0.999692,0.994091,0.993448,0.993433,0.946399
